# Funciones 

In [1]:
import numpy as np
import random
import math
import time 

"""Función para crear una solución aleatoria"""
def create_solution(items):
    dim = len(items) # Se obtiene el número de elementos
    elements = list(range(dim)) # Lista de elementos
    solution = np.zeros((dim,dim)) # Se crea una matriz de dimensiones dim x dim 
    
    for element in elements: # Se recorre cada uno de los elementos 
        solution[np.random.randint(dim)][element] = 1 # Se coloca el elemento en un contenedor de manera aleatoria
        
    return solution # Devuelve la solución creada


"""
# Función para convertir una solución a una solución factible
# Se considera las variables de decisión como una matriz binaria, filas = contenedores, columnas = elementos
# Toma como parametros: solution = matriz binaria de variables de desición, 
 items = pesos de los elementos, bin_capacity = capacidad de los contenedores
"""
def make_feasible(Solution, items, bin_capacity): 
    solution = Solution.copy()
    bins_capacity = solution @ items  # Se calculan las capacidades de cada contenedor de acuerdo a los elementos que contienen
    n = len(items) # Número de elementos 
    
    if any(element > bin_capacity for element in bins_capacity): # Pregunta si hay algún contenedor que revase su capacidad
        bins_full = np.where(bins_capacity > bin_capacity)[0] # Devuelve el índice de los contenedores que sobrepasaron su límite
        for bin_i in bins_full: # Se recorre cada contenedor que sobrepaso su límite 
            elements = np.nonzero(solution[bin_i])[0] # Indice de los elementos que se encuentran en ese contenedor (bin_i) lleno
            random.shuffle(elements) # Los índices de los elementos se ordenan de manera aleatoria
            
            for i in elements: # Se recorre sobre cada elemento del contenedor lleno (índices)
                lista = list(range(n)) # Lista de contenedores random (índices)
                random.shuffle(lista) # Se ordena al azar la lista de contenedores
                for j in lista: # Se recorre sobre los demás contenedores para poner el elemento indicado
                    if (np.sum(solution[j] @ items) + items[i]) <=  bin_capacity: # Verifica si el elemento entra en el nuevo contenedor
                        # En caso de entrar se realiza elintercambio del elemento al otro contenedor
                        solution[j,i] = 1 
                        solution[bin_i, i] = 0
                        break
                    else: 
                        # En caso de que el elemento no entre en el contenedor, se pasa a otro contenedor para probar
                        pass
                        
                if np.sum(solution[bin_i] @ items) <= bin_capacity: 
                    # Si el contenedor que estaba lleno ya no rebasa su capacidad, pasamos al siguiente contenedor lleno para variciarlo
                    break
                        
        return solution # Devuelve la solución

    else: # En caso de que no haya ningún contenedor que revase su capacidad en un principio, la solución ya es factible
        return solution # Devuelve la solución
    
"""Función para crear una población de soluciones factibles"""
def create_poblation_solution_feasible(items, n_poblation, bin_capacity): # items = pesos de los elementos, n_poblation = número de soluciones
    poblation = [] # Lista de la población
    
    for i in range(n_poblation): # Iterar para obtener una solución
        solution = create_solution(items) # Se crea solución aleatoria
        solution_feasible = make_feasible(solution, items, bin_capacity) # Se vuelve una solución factible 
    
        while any(np.array_equal(solution_feasible, element) for element in poblation):  # Se verifica que la solución ya no esté en la población
            # Se crea una nueva solución
            solution = create_solution(items)
            solution_feasible = make_feasible(solution, items, bin_capacity)
        poblation.append(solution_feasible) # Se añade la solución a la población
        
    return poblation # Devuelve la población factible

"""Función para evaluar la solución en la función objetivo"""
def evaluation_obj_function(solution):
    n_elements_bins = np.count_nonzero(solution, axis=1) # Vector binario que indica los contenedores con elementos (1) y sin elementos (0)
    bins_no_empty = np.count_nonzero(n_elements_bins) # Cuenta el número de contenedores con elementos
    return bins_no_empty # Devuelve el número de contenedores usados

"""Función para obtener la evaluación en la función objetivo de la población"""
def evaluation_poblation_obj_function(poblation):
    n = len(poblation) # Nümero de la población 
    evaluation = [] # Lista para guardar la evaluación de cada solución en la función objetivo
    for i in range(n): # Se recorre a cada solución
        evaluation.append(evaluation_obj_function(poblation[i])) # Se hace la evaluación y se guarda
    return evaluation # Retorna la evalución en la función objetivo


# Función One-first
def one_first(items, bin_capacity): # Metaheurística para encontrar buenas soluciones
    n = len(items) # Número de elementos
    #elements = sorted(range(len(items)), key=lambda i: items[i])
    elements = list(range(n)) # Lista del número de elementos
    random.shuffle(elements) # Se ordenada de forma aleatoria la lista de elementos
    solution = np.zeros((n,n)) # Matriz de solución inicializa en ceros de dimensión n x n
    bins = list(range(n))  #  Lista del número de contenedores
    
    for element in elements: # Se recorre cada elemento
        for bin_ in bins: # Se recorre cada contenedor
            if np.sum(solution[bin_] @ items) + items[element] <= bin_capacity: # Si el peso del contenedor bin + el peso del elemento element no supera su límite, entonces pone ese elemento en el contenedor
                solution[bin_, element] = 1 # Se agrega el elemento al contenedor
                break # Una vez agregado, pasamos al siguiente elemento
            else:
                pass # Si no entra en el contenedor entonces pasamos a otro contenedor 
    return solution # Regresa la solución 

# Función para crear una población de soluciones con one first
def one_first_poblation(n_poblation, items, bin_capacity):
    poblation_one_first = [] # Lista de la población de soluciones 
    solution = one_first(items, bin_capacity) # Se aplica la función de 
    
    for i in range(n_poblation): # Se itera el número de veces en que se desea crear la población
        while any(np.array_equal(solution, element) for element in poblation_one_first):  # Se verifica que la solución ya no esté en la población
            # Se crea una nueva solución
            solution = one_first(items, bin_capacity) # Se crea una solución mediante el método One first
        poblation_one_first.append(solution) # Se añade la solución a la población
    return  poblation_one_first # Devuelve la población de soluciones


"""Función para hacer la selección por torneo"""
def tournament_selection(poblation):
    n = len(poblation) # Número de elementos de la población 
    n_tournament = n // 3 # Número de la población del torneo, alrededor del 30%
    index_tournament_poblation = np.random.randint(0, n, n_tournament) # Indices aleatorios de las soluciones para el torneo
    val_obj_func = [] # Lista vacía para la evalución en la función objetivo
    
    for i in index_tournament_poblation: # Se recorre cada solución seleccionada para el torneo
        val_obj_func.append(evaluation_obj_function(poblation[i])) # Se agrega su evalución en la función objetivo
    
    elements_weight = zip(index_tournament_poblation, val_obj_func) # Se agrupan los índices de las soluciones con sus correspondientes valores en la función objetivo 
    elements_weight_sort = sorted(elements_weight, key=lambda x: x[1])  # x[1] representa el segundo elemento de la tupla, que es el peso, y los ordena en base a su peso
    elements_sort, weight_sort = zip(*elements_weight_sort) # Extrae los objetos y los pesos ordenados
    elements_sort = np.array(elements_sort) # Los elementos se convierten de tupla a vector
    weight_sort = np.array(weight_sort) # Los pesos se convierten de tupla a vector
    
    father1 = poblation[elements_sort[0]] # Mejor solución del torneo
    father2 = poblation[elements_sort[1]] # Segunda mejora solución del torneo

    return father1, father2 # Retorna las dos mejores soluciones

"""Función para hacer la cruza entre dos padres"""
def cruza(father1, father2):
    n = len(father1) # Número de elementos por cada padre
    lower_limit = math.floor(n * 0.3) # Límite inferior para realizar el corte
    up_limit = n - lower_limit # Límite superior para realizar el corte
    partition = random.randint(lower_limit, up_limit) # Se realiza la partición (parición binaria por columna) de manera aleatoria de acuerdo a los límites establecidos
    partitions = random.randint(0,1) # Se escoge un número al azar entre el 0 y el 1
    
    if partitions == 0: # Si el número al azar es 1 se realiza la primera configuración de hijos 
        son1 = np.concatenate((father1[:, :partition], father2[:, partition:]), axis=1)
        son2 = np.concatenate((father2[:, :partition], father1[:, partition:]), axis=1)
    else: # Si el número al azar es 2 se realiza la segunda configuración de hijos 
        son1 = np.concatenate((father2[:, :partition], father1[:, partition:]), axis=1)
        son2 = np.concatenate((father1[:, :partition], father2[:, partition:]), axis=1)
    
    if evaluation_obj_function(son1) >= evaluation_obj_function(son2): # Si el primer hijo de la solución es mejor que el segundo, se toma esta solución
        return son1
    else: # Si el segundo hijo resulta ser mejor se toma este último
        return son2
    
"""Función para realizar la muta"""
def muta(Solution, items, bin_capacity):
    solution = Solution.copy() # Se realiza una copia de la solución
    n = len(solution) # Número de elementos de la solución
    bins_no_zero = np.nonzero(np.any(solution, axis=1))[0] # Se guardan los contenedores no vacios
    bin_i = random.choice(bins_no_zero) # Se selecciona un contenedor al azar
    elements = np.nonzero(solution[bin_i])[0] # Se guardan los índices de los elementos dentro del contenedor
    element_i = random.choice(elements) # Se selecciona un elemento al azar
    lista = list(range(n)) # Lista del número de contenedores
    random.shuffle(lista) # Se ordena la lista de contenedores de manera aleatoria
    
    for j in lista: # Se recorre sobre los demás contenedores para poner el elemento indicado
        if (np.sum(solution[j] @ items) + items[element_i]) <=  bin_capacity: # Verifica si el elemento entra en el nuevo contenedor
            # En caso de entrar se realiza elintercambio del elemento al otro contenedor
            solution[bin_i, element_i] = 0
            solution[j,element_i] = 1 
            break
        else: 
            pass # En caso de no entrar en el contenedor se pasa al siguiente contenedore de la lista aleatoria
        
    return solution # Retorna la muta

"""Función para realizar el cambio del hijo al peor elemento de la población"""
def exchange(Poblation, son, Evaluation_obj_function):
    poblation = Poblation.copy() # Copia de la población
    evaluation = Evaluation_obj_function.copy() # Copia de la evaluación
    evaluation_son = evaluation_obj_function(son) # Evalución en la función objetivo
    worst_solution_i = np.max(evaluation) # Se obtiene el peor valor de la evaluación
    worst_solutions_i = np.where(evaluation == worst_solution_i)[0] # Lista de índices con la misma peor evaluación
    selec_worst_solution_i = random.choice(worst_solutions_i) # Se selecciona uno al azar de la lista de peores evaluaciones en la función objetivo
    poblation[selec_worst_solution_i] = son # Se sustituye la peor solución por el nuevo hijo
    evaluation[selec_worst_solution_i] = evaluation_son # Se sustiuye la evalución de la función objetivo de la peor solución por la del nuevo hijo 
    
    return poblation, evaluation # Retorna la población y la evaluación de la población en la función objetivo 
    
"""Algoritmo para solucionar el BPP implementando un algoritmo genético"""
def BPP_genectic_algorithm(items, bin_capacity, n_poblation, n_generations): # Se introducen los parámetros
    # items = lista de las cargas de los elementos, bin_capacity = capacidad máxima de los contenedores
    # n_poblation = número de población deseada, n_generations = número de generaciones
    poblation = create_poblation_solution_feasible(items, n_poblation, bin_capacity) # Se crea una población factible 
    evaluation = evaluation_poblation_obj_function(poblation) # Se obtiene la evaluación de la población creada en la función objetivo

    for i in range(n_poblation * n_generations): # Se actualiza la población hasta las generaciones deseadas 
        father1, father2 = tournament_selection(poblation) # Se obtienen dos padres por torneo
        son = cruza(father1, father2) # Se obtiene la cruza de los padres y se obtiene al mejor hijo
        son = make_feasible(son, items, bin_capacity) # Se realiza la factibilidad del hijo en caso de requerirlo 
        val_muta = random.randint(0,10) # Valor que determina si se realiza la muta
        if val_muta <= 5: # Condición que verifica si se realiza la mutación
            son = muta(son, items, bin_capacity) # Se realiza la mutación
            son = make_feasible(son, items, bin_capacity) # Se vuelve factible
        poblation, evaluation = exchange(poblation, son, evaluation) # Se intecarbia el nuevo hijo por la peor solución de la población

    return poblation, evaluation # Devuelve la población y su evaluación en la función objetivo

# Pruebas del algoritmo

## Intancia 1

In [21]:
# Se importan los pesos
with open('instancia1.txt', 'r') as file:
    items_list = file.readline().strip().split(',') # Lee la primera línea del archivo y convierte los números en una lista
    items = [int(element) for element in items_list] # Pesos de los elementos

star = time.time() # Tiempo de inicio

bin_capacity = 10 # Capacidad de los contenedores
n_poblation = 50 # Población
n_generations = 3000 # Generaciones

poblation, evaluation = BPP_genectic_algorithm(items, bin_capacity, n_poblation, n_generations) # Algoritmo genético

stop = time.time() # Tiempo final 

print('Evalución enla función objetivo: ', min(evaluation))
print(stop-star, 's')

Evalución enla función objetivo:  27
256.14182353019714 s


## Intancia 2

In [17]:
# Se importan los pesos
with open('instancia2.txt', 'r') as file:
    items_list = file.readline().strip().split(',') # Lee la primera línea del archivo y convierte los números en una lista
    items = [int(element) for element in items_list] # Pesos de los elementos

star = time.time() # Tiempo de inicio

bin_capacity = 10 # Capacidad de los contenedores
n_poblation = 50 # Población
n_generations = 3000 # Generaciones

poblation, evaluation = BPP_genectic_algorithm(items, bin_capacity, n_poblation, n_generations) # Algoritmo genético

stop = time.time() # Tiempo final 

print('Evalución enla función objetivo: ', min(evaluation))
print(stop-star, 's')

Evalución enla función objetivo:  61
129.61927032470703 s


## Intancia 3

In [27]:
# Se importan los pesos
with open('instancia3.txt', 'r') as file:
    items_list = file.readline().strip().split(',') # Lee la primera línea del archivo y convierte los números en una lista
    items = [int(element) for element in items_list] # Pesos de los elementos

star = time.time() # Tiempo de inicio

bin_capacity = 120 # Capacidad de los contenedores
n_poblation = 50 # Población
n_generations = 3000 # Generaciones

poblation, evaluation = BPP_genectic_algorithm(items, bin_capacity, n_poblation, n_generations) # Algoritmo genético

stop = time.time() # Tiempo final 

print('Evalución enla función objetivo: ', min(evaluation))
print(stop-star, 's')

Evalución enla función objetivo:  33
211.9282956123352 s


## Intancia 4

In [18]:
# Se importan los pesos
with open('instancia4.txt', 'r') as file:
    items_list = file.readline().strip().split(',') # Lee la primera línea del archivo y convierte los números en una lista
    items = [int(element) for element in items_list] # Pesos de los elementos

star = time.time() # Tiempo de inicio

bin_capacity = 120 # Capacidad de los contenedores
n_poblation = 50 # Población
n_generations = 3000 # Generaciones

poblation, evaluation = BPP_genectic_algorithm(items, bin_capacity, n_poblation, n_generations) # Algoritmo genético

stop = time.time() # Tiempo final 

print('Evalución enla función objetivo: ', min(evaluation))
print(stop-star, 's')

Evalución enla función objetivo:  55
127.68197965621948 s
